In [206]:
import json
import time
from datetime import datetime, timedelta
from sgp4.api import Satrec, jday, days2mdhms
from conj_gen import pos_tle
import numpy as np
import pandas as pd

There is a known conjunction between C1 (26463) and a GPS satellite, which occurred at 2022-03-28 11:28:34.215, with a miss distance of 6248m. This code finds the GPS satellite to check the accuracy of the TLE method.

https://www.space-track.org/basicspacedata/query/class/gp_history/EPOCH/2022-03-27--2022-03-29/PERIAPSIS/%3C36000/OBJECT_TYPE/PAYLOAD/LAUNCH_DATE/%3C2022-03-28/DECAY_DATE/null-val/distinct/true/orderby/NORAD_CAT_ID,EPOCH/predicates/norad_cat_id,epoch,launch_date,tle_line1,tle_line2/format/json

In [199]:
with open("c1_2022.json", 'r') as f:
    data = json.load(f)

print('Number of TLEs: ', len(data))

Number of TLEs:  41723


In [185]:
conj_time = datetime(2022,3,28,11,28,34,215000)
conj_time_julian = np.sum(jday(2022,3,28,11,28,34),axis=0)
norad = []
epoch = []
tle_line1 = []
tle_line2 = []
for e in data:
    norad.append(int(e['NORAD_CAT_ID']))
    epoch.append(datetime.strptime(e['EPOCH'], "%Y-%m-%dT%H:%M:%S.%f"))
    tle_line1 = np.append(tle_line1, e['TLE_LINE1'])
    tle_line2 = np.append(tle_line2, e['TLE_LINE2'])

norad_list = sorted(set(norad))

In [169]:
%%time

index = []
for i in norad_list:
    a = np.nonzero(np.array(norad) == i)
    epoch_temp = []
    for j in a[0]:
        epoch_temp.append(epoch[j])
    index.append(a[0][np.argmin(abs(np.array(epoch_temp) - conj_time))])

tle_store = []
for i in index:
    tle_store.append(tle_line1[i])
    tle_store.append(tle_line2[i])

print('Number of satellites: ', len(norad_list))
print('Number of saved TLEs: ', len(index))
print('Rows to process: ', len(tle_store))

Number of satellites:  7627
Number of saved TLEs:  7627
Rows to process:  15254
CPU times: total: 10.6 s
Wall time: 10.6 s


In [ ]:
%%time

# Time limits
sta = np.sum(jday(2022,3,28,11,25,0),axis=0)
end = np.sum(jday(2022,3,28,11,35,0),axis=0)
jd_range = np.arange(sta,end,1/86400)

# C1 TLE
index_c1 = np.argmin(abs(np.array(norad_list) - 26463))
sat_c1 = Satrec.twoline2rv(tle_store[2*index_c1],tle_store[2*index_c1+1])
r_c1 = np.zeros([jd_range.shape[0],3])
for i in range(jd_range.shape[0]):
    r_c1[i,:] = sat_c1.sgp4(jd_range[i],0)[1]

new_norad_list = []
min_dist = []
min_time = []
for i in range(len(index)):
    tle1 = tle_store[2*i]
    tle2 = tle_store[2*i+1]
    sat = Satrec.twoline2rv(tle1,tle2) # reading TLE
    r = np.zeros([jd_range.shape[0],3]) # initialising table of positions
    for j in range(jd_range.shape[0]):
        r[j,:] = sat.sgp4(jd_range[j],0)[1] # obtaining table of x y z
    new_norad_list.append(norad_list[i])
    dist = (np.sum(abs(r - r_c1)**2, axis=1))**0.5
    min_dist.append(np.min(dist))
    min_time.append(jd_range[np.argmin(dist)])
    
new_norad_list = np.array(new_norad_list)
min_dist = np.array(min_dist)
min_time = np.array(min_time)

In [204]:
results = pd.DataFrame({'NORAD': new_norad_list, 'dist-km': min_dist, 'time-diff-sec': abs(min_time - conj_time_julian)*86400})
results.loc[results['dist-km'] < 1000]

# temp = results['time-diff-sec'].to_numpy()
# print(np.max(temp))

,NORAD,dist-km,time-diff-sec
2060,26463,0.000000,214.000006
2241,28361,9.488946,2.998817


The GPS satellite was Navstar 55, or NORAD CAT ID 28361. The miss distance was calculated to be 9489m, some 3km off the true miss distance, and the TCA is off by 3 seconds.

In [208]:
# Checking which TLE has been selected

sat_id = 28361

b = np.nonzero(np.array(norad) == sat_id)[0]
for i in b:
    print(epoch[i])
    print(tle_line1[i])
    print(tle_line2[i])

old_index = np.argmin(abs(np.array(norad_list) - sat_id))
print(tle_store[2*old_index])
print(tle_store[2*old_index+1])

# Verifying my conjunction code works! 

t1 = pos_tle(tle_store[2*index_c1:2*(index_c1+1)],sta,end)
t2 = pos_tle(tle_store[2*old_index:2*(old_index+1)],sta,end)

print('Minimum distance: ', np.min(np.sum((t1-t2)**2,axis=1)**0.5))

2022-03-27 00:46:41.357568
1 28361U 04023A   22086.03242312 -.00000003  00000-0  00000-0 0  9998
2 28361  54.1513 149.2247 0143433 256.7830  96.1424  2.00564058130087
2022-03-28 15:07:42.572640
1 28361U 04023A   22087.63035385  .00000010  00000-0  00000+0 0  9991
2 28361  54.1517 149.1597 0143542 256.7838 169.8820  2.00558414130141
2022-03-28 15:07:42.572640
1 28361U 04023A   22087.63035385  .00000010  00000-0  00000-0 0  9992
2 28361  54.1517 149.1597 0143542 256.7838 169.8820  2.00558414130129
1 28361U 04023A   22087.63035385  .00000010  00000-0  00000+0 0  9991
2 28361  54.1517 149.1597 0143542 256.7838 169.8820  2.00558414130141
Minimum distance:  9.48894559649071
